In [1]:
%run "../../shared/utilz_includez.ipynb"

In [3]:
# https://www.kaggle.com/Cornell-University/movie-dialog-corpus ==> http://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html 

In [174]:
import re

In [49]:
dataset_dir = "../dataset/cornell_movie_dialogs_corpus"
sepz = '+++$+++'
listing_file='movie_titles_metadata.txt'
full_convoz_file = 'movie_lines.txt'
convoz_listing_file = 'movie_conversations.txt'
target_genres = ['romance', 'comedy', 'drama', 'sci-fi', 'mystery']

In [141]:
target_movies_list = {}
idx_target_movies_list = []

target_convoz_list = {}
idx_target_convoz_list = []

x_labelz = []
y_labelz = []

In [142]:
def is_within_year(year):
    return int(year.strip()[0:4] ) >= 2000 

def is_within_rating(imdbrating):
    return float(imdbrating.strip() ) >= 6

def is_within_genres( genres ):
    for g in genres:
        if g.strip() in target_genres:
            return True
    return False

def is_from_target_movie(mid):
    #print( f"'{mid}'" )
    return str(mid).strip() in idx_target_movies_list 

def turnStrToList(lines_list):#todo regex
    listz = str(lines_list).strip()[1:-2].replace('\'', '').split(',')
    #print( repr(listz) )
    return listz
    
def get_last_statement( lines_list ):     
    return turnStrToList( lines_list )[-1]


def get_first_statement( lines_list ): 
    return turnStrToList( lines_list )[0]

In [143]:
with open( f"{dataset_dir}/{listing_file}", 'r', encoding='iso-8859-1') as fd:
    for row in fd.readlines():
        # id, title, year, IMDB rating, # votes, genres_list 
        rec = row.split( sepz )
        # print( rec )
        
        if is_within_year( rec[2]) or \
            is_within_rating( rec[3]) or \
                is_within_genres( rec[5]):
            rid = f"{rec[0].strip()}" 
            target_movies_list[ rid ] = rec
            idx_target_movies_list.append( rid )
            
        
print( f"Found { len(target_movies_list) } suitable samples")
print( f"EG: { target_movies_list[ idx_target_movies_list[0]] }")
print( f"EG: {target_movies_list[ idx_target_movies_list[-1]] }")

Found 532 suitable samples
EG: ['m0 ', ' 10 things i hate about you ', ' 1999 ', ' 6.90 ', ' 62847 ', " ['comedy', 'romance']\n"]
EG: ['m616 ', ' zulu dawn ', ' 1979 ', ' 6.40 ', ' 1911 ', " ['action', 'adventure', 'drama', 'history', 'war']\n"]


In [ ]:
counter = 0
def parseConvo(rec, is_first=False, is_last=False ): 
    global counter
    #line_id, u_id, m_id, u_name, line_str
    rec = row.split( sepz )
    print( rec )
    if is_from_target_movie( rec[2] ):            
        #rid = f"{rec[0]}_{rec[1]}"        
        first_line = get_first_statement(rec[3])
        last_line = get_last_statement(rec[3])
        
        target_convoz_list[ rid ] = [ *rec[0:3], turnStrToList(rec[3]), first_line, last_line ]
        idx_target_convoz_list.append( rid )            
                
        if not is_last:
            x_labelz.append( last_line )
            
        if not is_first: 
            y_labelz.append( first_line )
    else:        
        print( f"NOT IN TARGET '{rec[2]}' :: {rec}" )
    
    print( f"XXX: {x_labelz}" )
    print(f"YYY: {y_labelz}\n")
    
    counter += 1
    
    if counter == 7:
        rec.trim()

In [157]:
x_labelz = []
y_labelz = []

counter = 0
with open( f"{dataset_dir}/{full_convoz_file}", 'r', encoding='iso-8859-1') as fd:    
    dcurrent, dnext = None, None     
    dcurrent = fd.readline()
    if dcurrent:
        dcurrent = dcurrent.split( sepz )
        while True:
        #line_id, u_id, m_id, u_name, line_str 
            dnext = fd.readline()
#             if counter == 7:
#                 break
            if dnext:
                dnext = dnext.split( sepz )
                
                if dnext[2].strip() == dcurrent[2].strip() and \
                    is_from_target_movie( dnext[2]) and \
                        dnext[1].strip() != dcurrent[1].strip(): 
                    x_labelz.append( dcurrent[4].strip())
                    y_labelz.append( dnext[4].strip())
            
#                 print(f">>>{dcurrent}\n>>>{dnext}" )                      
#                 print(f">>>{x_labelz}\n>>>{y_labelz}" )
                
                dcurrent = dnext
                dnext = None
                counter += 1
                
            else:
                break

print( f"Found { len(x_labelz) } ===  { len(y_labelz) } suitable samples")
print( f"EG: { x_labelz[0]} --> {y_labelz[0]}")
print( f"EG: { x_labelz[-1]} --> {y_labelz[-1]}")

Found 237283 ===  237283 suitable samples
EG: They do not! --> They do to!
EG: Good ones, yes, Mr Vereker. Gentlemen who can ride and shoot --> Colonel Durnford... William Vereker. I hear you 've been seeking Officers?


In [168]:
import tensorflow as tf
from tensorflow import keras

import nltk
#import textblob

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.model_selection import train_test_split 

In [196]:
def performanceReport(pred_y, target_y, title):
    print( f"Accuracy Score: { accuracy_score(target_y, pred_y)}\n")
    print( f"Classification Report: \n{ classification_report(target_y, pred_y)}\n")
    print( f"Confusion Matrix: \n { confusion_matrix(target_y, pred_y)}")
    
    
def preprocess(row):
    # remove punkts and numbers + make lower 
    res = str(row).lower()
    res = re.sub( "[^\w_\s\']+", '', res) ## keep possesive forms
    return res

def encode_bow(dset, ngramz=(1,1)):
    encoder = CountVectorizer(ngram_range=ngramz )
    encoded_matrix = encoder.fit_transform( dset )
    return encoded_matrix, encoder

def encode_tfidf(dset, ngramz=(1,1)):
    encoder = TfidfVectorizer(ngram_range=ngramz )
    encoded_matrix = encoder.fit_transform( dset )
    return encoded_matrix, encoder

def encode_with(dset, encoder):
    return encoder.transform( dset)

In [193]:
## train/test split << PS: can combine with vectorizer by passing preprocessin func
x_train, x_test, y_train, y_test = train_test_split( x_labelz, y_labelz, test_size=0.33, random_state=42)

print(f"Training Set: x, y = { len(x_train) }, { len(y_train) }")
print(f"Training Set EG.0: x, y = { x_train[0] }, { y_train[0] }")
print(f"Training Set EG.-1: x, y = { x_train[-1] }, { y_train[-1] }")

print(f"\nTesting Set: x, y = { len(x_test) }, { len(y_test) }")
print(f"Testing Set EG.0: x, y = { x_test[0] }, { y_test[0] }")
print(f"Testing Set EG.-1: x, y = { x_test[-1] }, { y_test[-1] }" )

Training Set: x, y = 158979, 158979
Training Set EG.0: x, y = Ah, er war gleich tot - I mean, sie war gleich tot - brauchen kein angst zu. Ein moment...Wart ein bissel...Wartein bissel. Fraulein Schmidt!, I was told that he did not die at once.
Training Set EG.-1: x, y = I know that., I didn't mean to hit you.

Testing Set: x, y = 78304, 78304
Testing Set EG.0: x, y = Did you tell Veronica?, She called me.
Testing Set EG.-1: x, y = A porco?, She's saying he's a-- a--  porco.


In [194]:
## preprocess
x_train = [ preprocess(x) for x in x_train ]
y_train = [ preprocess(x) for x in y_train ]
x_test = [ preprocess(x) for x in x_test ]
y_test = [ preprocess(x) for x in y_test ]
print(f"Training Set EG.0: x, y = { x_train[0] } ---> { y_train[0] }")
print(f"Training Set EG.-1: x, y = { x_train[-1] } ---> { y_train[-1] }")

print(f"Testing Set EG.0: x, y = { x_test[0] } ---> { y_test[0] }")
print(f"Testing Set EG.-1: x, y = { x_test[-1] } ---> { y_test[-1] }" )

Training Set EG.0: x, y = ah er war gleich tot  i mean sie war gleich tot  brauchen kein angst zu ein momentwart ein bisselwartein bissel fraulein schmidt ---> i was told that he did not die at once
Training Set EG.-1: x, y = i know that ---> i didn't mean to hit you
Testing Set EG.0: x, y = did you tell veronica ---> she called me
Testing Set EG.-1: x, y = a porco ---> she's saying he's a a  porco


In [199]:
## encode TFIDF 
encoded_x_train, encoder_x = encode_tfidf(x_train )
encoded_y_train = encode_with( y_train, encoder_x)
encoded_x_test = encode_with( x_test, encoder_x)
encoded_y_test = encode_with( y_test, encoder_x)
                                            
print(f"Training Set EG.0: x, y = { encoded_x_train[0] } " )
print(f"Testing Set EG.0: x, y = {encoded_x_test[0] }")


print(f"\nThe Encoder: {encoder_x}\n Vocab: { len(encoder_x.vocabulary_) }, {list(encoder_x.vocabulary_.keys())[:4] }")

Training Set EG.0: x, y =   (0, 33634)	0.18588479244259754
  (0, 14876)	0.18406789329022877
  (0, 4258)	0.2048299136163163
  (0, 4259)	0.21182199415651926
  (0, 24921)	0.21182199415651926
  (0, 12122)	0.39204186784748907
  (0, 45058)	0.2048299136163163
  (0, 1943)	0.21182199415651926
  (0, 20966)	0.2048299136163163
  (0, 5017)	0.21182199415651926
  (0, 34962)	0.17826556670046423
  (0, 23917)	0.08820275541507147
  (0, 39598)	0.4096598272326326
  (0, 15876)	0.4096598272326326
  (0, 43146)	0.24581162575677054
  (0, 12692)	0.14506471044873745
  (0, 1324)	0.125940887235991 
Testing Set EG.0: x, y =   (0, 44776)	0.17406030761620256
  (0, 42590)	0.8088469304347322
  (0, 38426)	0.40637461174700784
  (0, 10515)	0.38772326131705437

The Encoder: TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(

In [203]:
hidden_unitz = 128
vocab = list(encoder_x.vocabulary_.keys() ) 
model = keras.Sequential([
    keras.layers.Embedding( input_dim =len(vocab), output_dim=64 ),
    keras.layers.LSTM( hidden_unitz ),
    keras.layers.Dense( 10 )
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 64)          2884992   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 2,985,098
Trainable params: 2,985,098
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])